# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lihue,21.9789,-159.3672,28.77,61,75,8.23,US,1728520260
1,1,queenstown,-31.8976,26.8753,17.29,39,31,2.06,ZA,1728520349
2,2,waitangi,-43.9535,-176.5597,10.46,88,61,8.49,NZ,1728520350
3,3,le port,-20.9373,55.2919,21.98,86,52,1.08,RE,1728520351
4,4,kurumkan,54.3000,110.3000,-1.34,91,100,2.06,RU,1728520352


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
#%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
# YOUR CODE HERE
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_city_data_df = city_data_df.loc[city_data_df["Cloudiness"]==0, :]
# Drop any rows with null values
# YOUR CODE HERE
ideal_city_data_df.dropna(how="any")
# Display sample data
# YOUR CODE HERE
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,fort bragg,35.1390,-79.0060,17.73,81,0,0.00,US,1728520384
33,33,hawaiian paradise park,19.5933,-154.9731,29.29,68,0,5.14,US,1728520385
36,36,qamdo,31.1667,97.2333,6.82,57,0,1.41,CN,1728520389
39,39,hugo,45.1600,-92.9933,16.96,59,0,2.06,US,1728520392
44,44,kodiak,57.7900,-152.4072,7.73,36,0,5.66,US,1728520397
...,...,...,...,...,...,...,...,...,...,...
567,567,sanya,18.2431,109.5050,26.09,78,0,1.00,CN,1728520984
569,569,swan hill,-35.3378,143.5544,26.36,18,0,4.30,AU,1728520986
573,573,kiama,-34.6833,150.8667,21.88,63,0,2.68,AU,1728520991
580,580,sulphur,30.2366,-93.3774,23.98,39,0,2.06,US,1728520998


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ""
# Display sample data
# YOUR CODE HERE
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
32,fort bragg,US,35.1390,-79.0060,81,
33,hawaiian paradise park,US,19.5933,-154.9731,68,
36,qamdo,CN,31.1667,97.2333,57,
39,hugo,US,45.1600,-92.9933,59,
44,kodiak,US,57.7900,-152.4072,36,
62,huntingburg,US,38.2989,-86.9550,63,
78,carnarvon,AU,-24.8667,113.6333,58,
84,fisterra,ES,42.9049,-9.2629,81,
86,dezhou,CN,37.4486,116.2925,23,
92,tagiura,LY,32.8817,13.3506,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE
limit = 1
#filters = f"circle:{longitude},{latitude},{radius}"
#bias = f"proximity:{longitude},{latitude}"
params = {
    # YOUR CODE HERE
    "categories":"accommodation.hotel",
    #"filter":filters,
    "limit":limit,
    #"bias":bias,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"# YOUR CODE HERE
    params["bias"] = f"proximity:{longitude},{latitude}"# YOUR CODE HERE

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params) # YOUR CODE HERE

    # Convert the API response to JSON format
    name_address =  name_address.json() # YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(20)

Starting hotel search
fort bragg - nearest hotel: Airborne Inn Lodging
hawaiian paradise park - nearest hotel: No hotel found
qamdo - nearest hotel: 银杏商务宾馆
hugo - nearest hotel: Best Western Plus
kodiak - nearest hotel: No hotel found
huntingburg - nearest hotel: Quality Inn
carnarvon - nearest hotel: No hotel found
fisterra - nearest hotel: Hotel Temático do Banco Azul
dezhou - nearest hotel: No hotel found
tagiura - nearest hotel: No hotel found
pasni - nearest hotel: Juddi Hotel
naracoorte - nearest hotel: Kincraig Hotel /Smith & Jones Cafe
port lincoln - nearest hotel: Boston Hotel
sur - nearest hotel: Sur Hotel
gokwe - nearest hotel: No hotel found
birzebbuga - nearest hotel: Seabreeze
matoury - nearest hotel: La Marmotte
tazacorte - nearest hotel: App Leyma
laojunmiao - nearest hotel: No hotel found
selebi-phikwe - nearest hotel: Cresta Bosele Hotel
codrington - nearest hotel: No hotel found
candeleda - nearest hotel: Hotel Rural el Camino
ontario - nearest hotel: Travelodge Onta

,City,Country,Lat,Lng,Humidity,Hotel Name
32,fort bragg,US,35.1390,-79.0060,81,Airborne Inn Lodging
33,hawaiian paradise park,US,19.5933,-154.9731,68,No hotel found
36,qamdo,CN,31.1667,97.2333,57,银杏商务宾馆
39,hugo,US,45.1600,-92.9933,59,Best Western Plus
44,kodiak,US,57.7900,-152.4072,36,No hotel found
62,huntingburg,US,38.2989,-86.9550,63,Quality Inn
78,carnarvon,AU,-24.8667,113.6333,58,No hotel found
84,fisterra,ES,42.9049,-9.2629,81,Hotel Temático do Banco Azul
86,dezhou,CN,37.4486,116.2925,23,No hotel found
92,tagiura,LY,32.8817,13.3506,70,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
#%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols= ["Hotel Name", "Country"]

)

# Display the map
# YOUR CODE HERE
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)